<a href="https://colab.research.google.com/github/sdgroeve/ML-course-VIB-2020/blob/master/notebooks/Histone_marks_PyCaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Histone modifications

In [120]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

random_seed = 123
np.random.seed(random_seed)

# 1. Reading the data

In [121]:
import pandas as pd

train = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test.csv")
test_labels = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/ML-course-VIB-2020/master/data/data_test_label.csv")

In [122]:
from sklearn import preprocessing

scaler_minmax = preprocessing.MinMaxScaler()

train_label = train.pop("Label")
train_ids = train.pop("GeneId")
test_ids = test.pop("GeneId")

scaler_minmax.fit(train)
train_norm = pd.DataFrame(scaler_minmax.transform(train),columns=train.columns)
test_norm = pd.DataFrame(scaler_minmax.transform(test),columns=test.columns)

test_norm["Label"] = test_labels["Label"]
train_norm["Label"] = train_label

In [123]:
! pip install pycaret

In [124]:
from pycaret.classification import *

In [125]:
#models()

In [126]:
setup(data=train_norm,target="Label",numeric_features=list(train.columns)[:-1], 
      train_size=0.8, silent=True, use_gpu=False, test_data=test_norm, preprocess=False)

,Description,Value
0,session_id,3595
1,Target,Label
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(10436, 501)"
5,Missing Values,False
6,Numeric Features,500
7,Categorical Features,0
8,Transformed Train Set,"(10436, 500)"
9,Transformed Test Set,"(5049, 500)"


(False,
 None,
 True,
 'lightgbm',
 None,
 [],
 'Label',
 {'acc': <pycaret.containers.metrics.classification.AccuracyMetricContainer at 0x7f8e81a38d30>,
  'auc': <pycaret.containers.metrics.classification.ROCAUCMetricContainer at 0x7f8e81a38df0>,
  'recall': <pycaret.containers.metrics.classification.RecallMetricContainer at 0x7f8e85ce0520>,
  'precision': <pycaret.containers.metrics.classification.PrecisionMetricContainer at 0x7f8e85ce0b20>,
  'f1': <pycaret.containers.metrics.classification.F1MetricContainer at 0x7f8e81a43340>,
  'kappa': <pycaret.containers.metrics.classification.KappaMetricContainer at 0x7f8e81a431f0>,
  'mcc': <pycaret.containers.metrics.classification.MCCMetricContainer at 0x7f8e81a43430>},
 False,
 False,
 0       1
 1       0
 2       0
 3       0
 4       1
        ..
 5044    1
 5045    1
 5046    1
 5047    0
 5048    0
 Name: Label, Length: 5049, dtype: int64,
 'lightgbm',
 False,
 [],
 5,
 {'lr': <pycaret.containers.models.classification.LogisticRegression

In [127]:
from sklearn.metrics import log_loss

add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False, target="pred_proba")

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f8eca866040>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [128]:
#compare_models(cross_validation=False,sort="LogLoss")

In [129]:
lr = create_model("lr",fold=5)
#lightgbm = create_model("lightgbm",folds=5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.8587,0.9152,0.8495,0.8669,0.8581,0.7175,0.7176,0.3774
1,0.8548,0.9121,0.8419,0.8658,0.8537,0.7097,0.7099,0.3800
2,0.8424,0.9098,0.8467,0.8411,0.8439,0.6847,0.6847,0.3892
3,0.8414,0.9039,0.8505,0.8369,0.8436,0.6828,0.6828,0.4004
4,0.8591,0.9174,0.8438,0.8720,0.8577,0.7183,0.7187,0.3681
Mean,0.8513,0.9117,0.8465,0.8565,0.8514,0.7026,0.7028,0.3830
SD,0.0078,0.0047,0.0033,0.0145,0.0064,0.0157,0.0158,0.0110


In [130]:
from pycaret.internal.utils import (
    param_grid_to_lists,
    get_logger,
    get_class_name,
    np_list_arange,
)

tune_grid = {
            "num_leaves": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200],
            "learning_rate": np_list_arange(0.001, 0.5, 0.001, inclusive=True),
            "n_estimators": np_list_arange(10, 300, 10, inclusive=True),
            "min_split_gain": [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
            "reg_alpha": [
                0.0000001,
                0.000001,
                0.0001,
                0.001,
                0.01,
                0.0005,
                0.005,
                0.05,
                0.1,
                0.15,
                0.2,
                0.3,
                0.4,
                0.5,
                0.7,
                1,
                2,
                3,
                4,
                5,
                10,
            ],
            "reg_lambda": [
                0.0000001,
                0.000001,
                0.0001,
                0.001,
                0.01,
                0.0005,
                0.005,
                0.05,
                0.1,
                0.15,
                0.2,
                0.3,
                0.4,
                0.5,
                0.7,
                1,
                2,
                3,
                4,
                5,
                10,
            ],
            "feature_fraction": np_list_arange(0.4, 1, 0.1, inclusive=True),
            "bagging_fraction": np_list_arange(0.4, 1, 0.1, inclusive=True),
            "bagging_freq": [1, 2, 3, 4, 5, 6, 7],
            "min_child_samples": np_list_arange(5, 100, 5, inclusive=True),
        }

#tuned_lightgbm = tune_model(lightgbm, optimize="LogLoss", tuner_verbose=1, n_iter=10,
#                            custom_grid=tune_grid)
tuned_lr = tune_model(lr, optimize="LogLoss", tuner_verbose=1, n_iter=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,0.8678,0.9194,0.8610,0.8743,0.8676,0.7356,0.7357,0.3693
1,0.8467,0.9100,0.8343,0.8571,0.8456,0.6935,0.6938,0.3883
2,0.8487,0.9110,0.8495,0.8495,0.8495,0.6973,0.6973,0.3806
3,0.8506,0.9125,0.8305,0.8668,0.8482,0.7012,0.7018,0.3828
4,0.8391,0.9117,0.8305,0.8466,0.8385,0.6782,0.6783,0.3868
5,0.8477,0.9059,0.8552,0.8440,0.8496,0.6954,0.6954,0.3990
6,0.8456,0.9108,0.8438,0.8487,0.8462,0.6913,0.6913,0.3844
7,0.8476,0.8991,0.8629,0.8389,0.8507,0.6950,0.6953,0.4133
8,0.8552,0.9217,0.8400,0.8681,0.8538,0.7105,0.7109,0.3556
9,0.8591,0.9154,0.8419,0.8735,0.8574,0.7182,0.7187,0.3744


In [131]:
#final_lightgbm = finalize_model(tuned_lightgbm)
#final_lr = finalize_model(tuned_lr)

In [132]:
#result = predict_model(final_lightgbm,data=test_norm)
result = predict_model(tuned_lr,data=test_norm)
#result = predict_model(tuned_lr)

In [133]:
tuned_lr

LogisticRegression(C=1.056, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=3595, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [134]:
from sklearn.metrics import log_loss

print(log_loss(test_norm["Label"],list(result["Score"])))

1.2494401742151229


In [135]:
"""
from sklearn.linear_model import LogisticRegression

cls = LogisticRegression()

train_y = train_norm.pop("Label")
train_X = train_norm
test_y = test_norm.pop("Label")
test_X = test_norm

cls.fit(train_X,train_y)

predictions = cls.predict_proba(test_X)

print(log_loss(test_y,predictions[:,1]))
"""

'\nfrom sklearn.linear_model import LogisticRegression\n\ncls = LogisticRegression()\n\ntrain_y = train_norm.pop("Label")\ntrain_X = train_norm\ntest_y = test_norm.pop("Label")\ntest_X = test_norm\n\ncls.fit(train_X,train_y)\n\npredictions = cls.predict_proba(test_X)\n\nprint(log_loss(test_y,predictions[:,1]))\n'